In [1]:
from google.cloud import storage
import os
import csv
import random
import numpy as np
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.models import Model, load_model
from keras.applications.resnet50 import ResNet50
import keras.metrics as metrics
from keras.layers import Average, Input
import numpy as np
import glob
import errno

BLACKLIST = ['LAUIHISOEZIM5ILF',
             '2018050121043822',
             '2018050120260258']

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def ensemble(models, model_input):
    outputs = [model(model_input) for model in models]
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [3]:
def get_ensemble_model():
    model_base = ResNet50(weights='imagenet', include_top=False)
    model_1 = load_model('/home/harold_triedman/elvo-analysis/sandbox/stage_1_resnet_v7_keep')
    model_1 = Model(inputs=model_base.input,
                    outputs=model_1(model_base.output))
    model_2 = load_model('/home/harold_triedman/elvo-analysis/sandbox/stage_1_resnet_v8_hold')
    model_2 = Model(inputs=model_base.input,
                    outputs=model_2(model_base.output))
    model_3 = load_model('/home/harold_triedman/elvo-analysis/sandbox/stage_1_resnet_v9_keep')
    model_3 = Model(inputs=model_base.input,
                    outputs=model_3(model_base.output))

    models = [model_1, model_2, model_3]
    model_input = Input(shape=(220, 220, 3))
    ensembleModel = ensemble(models, model_input)
    return ensembleModel

In [4]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


metrics.sensitivity = sensitivity
metrics.specificity = specificity

In [5]:
ensembleModel = get_ensemble_model()
ensembleModel.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])
ensembleModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 220, 220, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            26736513    input_2[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 1)            26736513    input_2[0][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 1)            26736513    input_2[0][0]                    
__________________________________________________________________________________________________
average_1 

In [6]:
!gsutil rsync -d -r gs://elvos/mip_data/multichannel/ /home/harold_triedman/elvo-analysis/data/multichannel

Building synchronization state...
Starting synchronization...
Copying gs://elvos/mip_data/multichannel/axial/0RB9KGMO90G1YQZD.npy...
Copying gs://elvos/mip_data/multichannel/axial/2KMKXR2G1BLD0C2G.npy...          
Copying gs://elvos/mip_data/multichannel/axial/3AWM4ZZHCWJ8MREY.npy...          
Copying gs://elvos/mip_data/multichannel/axial/56GX2GI8AGT0BIHN.npy...          
/ [4 files][  6.3 MiB/  6.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://elvos/mip_data/multichannel/axial/5H94IH9XGI83T610.npy...
Copying gs://elvos/mip_data/multichannel/axial/5KZSOKNYS84ZTDK6.npy...          
Copying gs://elvos/mip_data/multichannel/axial/6BMRRS9RAZUPR3IL.npy...          
Copying gs://elvos/mip_data/multichannel/axial/8TLM2DUBYEE

In [8]:
FAILURE_ANALYSIS = ['SSQSB70QYC5L5CJJ',
                    'MTWW42SPCGLHEDKY',
                    'GKDV3FW4M56I3IKV',
                    'BMFSUHVBPJ7RY56P',
                    'NWO33W453F6ZJ4BU',
                    '8TLM2DUBYEE2GDH0',
                    'J3JHPC3E15NZGX34',
                    'J3JHPC3E15NZGX35',
                    '3AWM4ZZHCWJ8MREY',
                    'MHL54TPCTOQJ2I4K',
                    '5H94IH9XGI83T610',
                    'UGXVSPJLHJL6AHSW',
                    '2KMKXR2G1BLD0C2G',
                    'PCNMFAZL5VWWK7RP',
                    'VVGO45TQNOASBLZM',
                    'NHXCOHZ4HH53NLQ6',
                    'F8W2RDY3D6L2EOFT',
                    'KK2Y9XHUUUC5LISA',
                    'HZLBHRHYLSY9TXJ4',
                    '0RB9KGMO90G1YQZD',
                    'J2JPFK8ZOICHFG34',
                    'HLXOSVDF27JWNCMJ',
                    'AEPRN5R7W2ASOGR0',
                    '99YJX0CY4FHHW46S',
                    'LOZOKQFJMDLL6IL5',
                    'STCSWQHX4UN23CDK',
                    'ZC9H37RWIQ90483S',
                    'CJDXGZHXAGH7QL3C',
                    '5KZSOKNYS84ZTDK6',
                    'AKZ8T688ZRU9UTY2',
                    '6BMRRS9RAZUPR3IL',
                    'HXLMZWH3SFX3SPAN',
                    'GHVG2CNNRZ65UBEU',
                    'TSDXCC6X3M7PG91E',
                    'IP4X9W512RO56NQ7',
                    'LNU3P20QOML7YGMZ',
                    '56GX2GI8AGT0BIHN',
                    'TSZFE43KG3NQJR69',
                    'IXRSXXZI0S6L0EJI',
                    'FCYGZ75WMW6L4PJM',
                    'KOE9CU24WK2TUQ43',
                    'KOE9CU24WK2TUQ44',
                    'TFIG39JA77W6USD3',
                    'XQBRGW3CYGNUMWHI',
                    'PB7YJZRJU74HFKTS',
                    'EUNTRXNEDB7VDVIS',
                    'JQWIIAADGKE2YMJS',
                    'NXLFQLVZRLUEK2UF',
                    'RHWTMTHC7HIWZ2YZ',
                    'RW13S0OR03CO7OP5',
                    'LYUO2OPTNYUBCHT2',
                    'CSCIKXOMNAIB3LUQ',
                    'Q8BNE59JIKQLLYJ1',
                    'K2GS9PIQ1E0DBDBE',
                    'WWEFFBIMLZ3KLQVZ']

In [14]:
path = '/home/harold_triedman/elvo-analysis/data/multichannel/axial/*.npy'
files = glob.glob(path)
predictions = []
for name in files:
    file_id = name.split('/')[7]
    file_id = file_id[:-4]
    print(file_id)
    if file_id in BLACKLIST or file_id not in FAILURE_ANALYSIS:
        continue
    
    prediction = [file_id]
    
    try:
        to_predict = np.load(name)
        to_predict = np.transpose(to_predict, (1, 2, 0))
        dims = np.shape(to_predict)
        to_predict = zoom(to_predict, (220 / dims[0],
                                       220 / dims[1],
                                       1))
        to_predict = to_predict[np.newaxis, :, :, :]
        probability = ensembleModel.predict(to_predict)
        probability = np.squeeze(probability)
        prediction.append(probability.tolist())
        predictions.append(prediction)
        
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

IAUKV5R644JZFD55
OI623KTR79Z90BG1
DRS5DZGN5PVUZX4Y
P64MELY8IJSLJ1WW
0MTDDGCF20DKOR59
VKMYOQD5A64UFEED
DVYKX4XMBHD57MZJ
TFCLOHN4ROEYF6QY
YDJX25T27CYATQ25
43ERS29KX3LPF334
PAIDKQGSUI0KZ54S
PKDK1R3JGEDSY9T2
NMHOGEKTK7LT57ZI
DHG4RIY13DFISJ35
27QV9Z81JZB09QPH
QPDX2K3DS7IS5QNM
RMID4XEKTVEYCSGK
PYMCG60E4FESH559
5D2RFZWIZ8G6K1HX
G8EI396DZCRABG20
A56W94SJCRL9BR9P
MSS82V0V6T9P13CV
AJHYOJIG3DZLSIKI
WKKADQ0Q6519IXWY
FYSSUUSC5XYNLUQW
XZUUUC426GUXSCGJ
M8PXGLRLDQUQ9497
K3C28ASFAF7ZLEIH
6GA622A55T3AOGTL
UUKSYXBYQQZATBP5
ANZM4SIQFTWG7K47
9DWQA9BENM1XBI71
QIA74LHB1XWMP523
WHNAL8GW8TJ8Y4AK
GTSAGN2MRE5D0AZI
Z3AINLH4Y07ITBRR
YSEPTJX4TY1WQY1T
PBR2STY62FZ3MQHZ
9UFGGHS7R1O40ZU9
FPTFJD4JZA7ZKYQJ
XPBRBUB6YYHIWHVO
CF7JC6FO4N0S96KN
UYA5YJUAVC3SE6HV
5M81E9S7W99ZAY7K
RYNCGNVRZH2LB7BN
SP73CHWDY57ND02Z
UO04YVZ0X59PBH6A
9AF84HK0K5CG471Y
SEV2GAEFNJBLX3ZJ
GCSZPU5LPJII8FGU
S6YJ44EEBN5T211U
3LEMIRZE3FJWTWOT
SZXTZ3LQ5MBFDFXR
KPC2RVJBDHQLC5EM
IJBSBL7IRLNRVXUC
64V9V1DQO6O3JJ8F
IETROE62UNT5QVWS
C67K82FMKN435ZNB
ZAJH6DQQBAECJ7

WMMP0B4WE4H12UL5
YOZ19SX2BIALZHYO
AEPRN5R7W2ASOGR0
OOAKLUZ2A4XYXC4X
NWRLBSNF3RVU3MAC
S1MFJS5SSJ12LGEF
ZTFA5BJQHNJIRH4T
MEP2WAB3JNIXJTON
72IYRWNFFY7OFPJ7
26UJHQ45YJOBI1WA
6Y2HYY248E0T6HLO
IIOOPK6T2ZWDZMU3
Q7TZ7D9MDFNNTUIE
UI0XEE9RCAEFSYP2
OGRRL3DQAJGEL0ZH
OJFOT7U85BBO92CL
SWSY42N45J3YWV5I
ILO1WFJV843ZHAME
JAANEAKBEZPKV4JV
PMUXVQHYYYBOK4RL
NLXOSUXOZZHPFQQX
PYZP7WQ4QRH46EFS
MKBXWA7XQVFZRVRK
ADVBNYHQ77F3AIMR
3SFAQXXE8MAOTWYL
20180501210438227
NHCZX3GOEMPDDHW3
QHD8C9LSCKNYCRVH
J2JPFK8ZOICHFG34
QJ9YG0LHX1SFOKLN
CCGEZOFGLKC3USJ8
4UXRRH5MVZVQ1HQW
H4C8M2RF0ASI8M5R
RLZKSYKP7FOKRC1O
PVMXD3RZSHEQTYW3
UGAYD4C2EMSUZ7LV
GFFSQZ18XGOVEET0
6OWGYLAAJCBPWEZ4
OLIQUZ7ERFP4DJVD
Q0YMNAR31STBFPG6
DMYPNOEZ9IWGA8AV
VTAJ4O0QLLZ4KVD2
LAZFQV32NQC0S558
IGIIMA3LABMNETMA
RKIZJRFZGCXD4EOL
NXDFBWPXYWHEIO2O
ZANU4GPPZNIFRN6W
DZTDN47H2IKSYN3Q
89U2QG6ISBDM51GI
EGCV7CJ6LF2CACQM
MJFHYCVEV2GTJD3N
MKKOO7NDN26O6RLZ
XSNEILWLDG2HVSVH
60BQLN9SD3IXJ5AI
JCMIIGHKFDRU911D
WYMX6QXJXTECQ81F
RM083BCXKKZQOR9Y
JTGK554CWBB6JDQJ
ZX3B62KA1WW46

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


GJ5CHFO5E33QWWNW
DSIFGAA3BMQKFE7A
37ZPX63RET9P2QHM
IP4X9W512RO56NQ7
5FY8Y34MBL26F0EE
OQWBV4PPVHOOYRWI
UOSAPEMY76KAGOUR
EURBSIKPYU66GBRW
82KAG7KXLABZCMKJ
JT6N68SSZWVYS4NF
EH3P3JW1V3L8GNJ7
6XB4FHWAXDVQ79NK
BMTJZEMYGJCVHX7N
YRPLT2NPTAFJ25RR
DXLPP6QTNOHQRHVG
I5LGG0BM4IP16DPT
VJNCTEEYR7U4HIQU
8LWUQKCNESCFMX7J
3YQNRP7QKLAD6YN5
KHPDR2WKRBY6ETGF
SE3O4E4F4Z70C75B
GXR5YMJB3PZQNLAC
L6JGZ08WAJCUWRSH
KOM02ZTT5IZQ41QL
XJLRFX244JGJZUUF
CGQFS5PB627OWLL7
SXF2FCCYEQLXMQO5
B63PZNF524OFKIZT
NEYLUIYPCL6O74O6
GBA7XJ5E7ITWZW7K
9G0Y9ORHYQF87HGL
W098UN5Y50II02BX
HIAEQWMTUGATOAVI
OLCBKEY2WKCWPHJW
ECZZ9D6T0D9K48X6
M1ZMVHI4X7C5J4OX
JNTL5ESEIA5FWMUQ
ML05G1O6XMYNN1JE
3806JEDXUNBDXJOT
FUVL4PLSUHRCEAHC
FBGMN3O08GW5GG91
NXLFQLVZRLUEK2UF
PHE2B59NIQZC67UJ
2EU9CLTGPI4V6XUD
UBDRB3DMSB6TJ76N
5NW5T7JL1UGKA9ZW
PQ918CDGQIFXXXN1
NJPOTXDCGONCOU7C
QA48KZ7WBFX9ZOIA
VRSBMSXUYCQ3NC5A
CPH2XKL5E8X02V51
LQG4BRZPSCMR5XAN
LOZOKQFJMDLL6IL5
Z9F4JC7DA3ZIW46J
IWU76ETUE9UIZN8O
YVVX9JSQIXHS3JC5
DNO6CS3YNGFMUWXL
TFV4472GKPYD269N
JYI7KFRTX8J9V5

In [15]:
print(predictions)

[['TFIG39JA77W6USD3', 0.9004835486412048], ['3AWM4ZZHCWJ8MREY', 0.31031009554862976], ['8TLM2DUBYEE2GDH0', 0.40326058864593506], ['TSZFE43KG3NQJR69', 0.9976681470870972], ['GHVG2CNNRZ65UBEU', 0.31350043416023254], ['TSDXCC6X3M7PG91E', 0.5941945314407349], ['NWO33W453F6ZJ4BU', 0.9697836637496948], ['NHXCOHZ4HH53NLQ6', 0.46970826387405396], ['LNU3P20QOML7YGMZ', 0.9991475939750671], ['FCYGZ75WMW6L4PJM', 0.3361055254936218], ['PCNMFAZL5VWWK7RP', 0.8671993613243103], ['56GX2GI8AGT0BIHN', 0.3199588656425476], ['6BMRRS9RAZUPR3IL', 0.794684648513794], ['RW13S0OR03CO7OP5', 0.9999966621398926], ['PB7YJZRJU74HFKTS', 0.34623464941978455], ['5H94IH9XGI83T610', 0.948772668838501], ['GKDV3FW4M56I3IKV', 0.0044886087998747826], ['UGXVSPJLHJL6AHSW', 0.9008006453514099], ['99YJX0CY4FHHW46S', 0.7025017738342285], ['LYUO2OPTNYUBCHT2', 0.3129887580871582], ['Q8BNE59JIKQLLYJ1', 0.014681002125144005], ['ZC9H37RWIQ90483S', 0.010995323769748211], ['AEPRN5R7W2ASOGR0', 0.6831614971160889], ['J2JPFK8ZOICHFG34', 0.

In [16]:
import pandas as pd

In [17]:
cols = ["ID", "Prob"]
preds_df = pd.DataFrame(data=predictions, columns=list(cols))

In [18]:
print(preds_df)

                  ID      Prob
0   TFIG39JA77W6USD3  0.900484
1   3AWM4ZZHCWJ8MREY  0.310310
2   8TLM2DUBYEE2GDH0  0.403261
3   TSZFE43KG3NQJR69  0.997668
4   GHVG2CNNRZ65UBEU  0.313500
5   TSDXCC6X3M7PG91E  0.594195
6   NWO33W453F6ZJ4BU  0.969784
7   NHXCOHZ4HH53NLQ6  0.469708
8   LNU3P20QOML7YGMZ  0.999148
9   FCYGZ75WMW6L4PJM  0.336106
10  PCNMFAZL5VWWK7RP  0.867199
11  56GX2GI8AGT0BIHN  0.319959
12  6BMRRS9RAZUPR3IL  0.794685
13  RW13S0OR03CO7OP5  0.999997
14  PB7YJZRJU74HFKTS  0.346235
15  5H94IH9XGI83T610  0.948773
16  GKDV3FW4M56I3IKV  0.004489
17  UGXVSPJLHJL6AHSW  0.900801
18  99YJX0CY4FHHW46S  0.702502
19  LYUO2OPTNYUBCHT2  0.312989
20  Q8BNE59JIKQLLYJ1  0.014681
21  ZC9H37RWIQ90483S  0.010995
22  AEPRN5R7W2ASOGR0  0.683161
23  J2JPFK8ZOICHFG34  0.009833
24  AKZ8T688ZRU9UTY2  0.018785
25  RHWTMTHC7HIWZ2YZ  0.595081
26  MTWW42SPCGLHEDKY  0.017844
27  CJDXGZHXAGH7QL3C  0.985973
28  VVGO45TQNOASBLZM  0.193791
29  HLXOSVDF27JWNCMJ  0.341132
30  0RB9KGMO90G1YQZD  0.537154
31  WWEF

In [19]:
preds_df.to_csv('/home/harold_triedman/elvo-analysis/ids_preds_new.csv', index=False)